In [1]:
import numpy as np
import pandas as pd
import time
from gensim.models import Word2Vec as wv
from gensim.models.word2vec import LineSentence
from scipy import sparse
from collections import defaultdict
from sklearn import svm

C:\Users\iagoa\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def leitura_arquivo(path):
    f1 = open(path, 'r', encoding='utf8')
    text = []
    tags = []
    word_tags = defaultdict(list)
    #leio arquivo e crio uma lista [word,tag]
    for line in f1:
        tokens = line.split()
        for token in tokens:
            word =  token.split('_')[0]
            tag =  'POS_'+token.split('_')[1]
            text.append(word)
            tags.append(tag)
            if tag not in word_tags[word]:
                word_tags[word].append(tag)

    return text,tags, word_tags

def words_to_num(text):
    words = {text[0][0]: 0, text[1][0]: 0}
    tags = {text[0][1]: 0, text[1][1]: 0}
    
    for i in range(2,len(text)):
        words[text[i][0]] = 0
        tags[text[i][1]] = 0
    
    #agora tenho um número único para cada palavra/tag
    return np.array(list(words.keys()) + list(tags.keys()))

def cria_dict_num(num_of_words):
    num_dict = {}
    for i in range(len(num_of_words)):
        num_dict[num_of_words[i]] = i
    return num_dict

In [3]:
%%time
path = 'macmorpho-train.txt'
text,tags, word_tags = leitura_arquivo(path)
text = np.array(text)

Wall time: 1.95 s


In [4]:
%%time
model = wv(text.reshape(-1,1), size = 1500, min_count=1, window=10, iter=10)

Wall time: 1min 17s


In [5]:
X_train = []
for palavra in model.wv.vocab:
    X_train.append(model.wv[palavra])

In [6]:
tag_dict = {}
word_dict = {}
i = 0
j = 0
for tag in tags:
    tag_dict[tag] = 0
for tag in tag_dict.keys():
    tag_dict[tag] = i
    i += 1
for word in model.wv.vocab:
    word_dict[word] = j
    j += 1
vec_tag = np.array(list(tag_dict.keys()))
vec_word = np.array(list(model.wv.vocab))

In [7]:
mtx = sparse.lil_matrix((len(vec_word), len(vec_tag)))

In [8]:
%%time
for i in range(len(text)):
    mtx[word_dict[text[i]],tag_dict[tags[i]]] = 1

Wall time: 10.4 s


In [9]:
Y_train = []
for i in range(len(vec_word)):
    Y_train.append(mtx[i,1])

In [10]:
'''%%time
clf = svm.SVC()
clf.fit(X_train,Y_train)'''

'%%time\nclf = svm.SVC()\nclf.fit(X_train,Y_train)'

In [11]:
'''teste = clf.score(X_train[:150], Y_train[:150])
teste'''

'teste = clf.score(X_train[:150], Y_train[:150])\nteste'

In [10]:
%%time
X = X_train[:20000]
y = Y_train[:20000]

Wall time: 1 ms


In [11]:
%%time
svr_rbf = svm.SVC(kernel='rbf', C=1e3, gamma=0.1)
#svr_lin = svm.SVC(kernel='linear', C=1e3)
#svr_poly = svm.SVC(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(X, y)#.predict(X)
#y_lin = svr_lin.fit(X, y)#.predict(X)
#y_poly = svr_poly.fit(X, y)#.predict(X)

Wall time: 19min 30s


In [14]:
%%time
clf = svm.SVC()
clf.fit(X,y)

Wall time: 19min


In [ ]:
'''print(y_lin.score(X,y),
y_poly.score(X,y),
y_rbf.score(X,y))'''

In [ ]:
#print(y_poly.score(X,y))

In [ ]:
#print(y_lin.score(X,y))

In [16]:
print(clf.score(X,y))

0.60045


In [12]:
print(y_rbf.score(X,y))

0.78415
